In [11]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys
import pickle

In [12]:
#PARAMS
p = 127
Fp = GF(p)
n = 128
k = 76
w = 80
r = n-k
t = n
z = 7
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])

In [13]:
#FUNZIONI DI UTILITA

def generate_random_seed():
    set_random_seed()
    return initial_seed()

def set_seed(seed):
    set_random_seed(seed)
    return initial_seed()

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

def convert_int_to_hash_string(input):
    return input.to_bytes((input.bit_length() + 7) // 8, 'big').hex()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    set_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    set_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed=0, eta=[]):
    e = vector(Fp, n)
    if len(eta)==0:
        eta = create_random_vector(Fz, n, seed)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def create_vector_over_Fp_star(Fp_star, t, seed):
    set_random_seed(seed)
    return [Fp_star.random_element() for i in range(t)]

def generate_vector_fixed_hamming_w(w, t, seed):
    random.seed(seed)
    vector = [0] * t
    indices = random.sample(range(t), w)
    for index in indices:
        vector[index] = 1
    return vector

def component_wise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

def convert_object_to_binary_string(input):
    # Serializza l'oggetto in un formato binario
    binary_data = pickle.dumps(input)
    # Converte la stringa binaria in una rappresentazione di stringa binaria
    binary_string = ''.join(format(byte, '08b') for byte in binary_data)
    return binary_string

def get_object_from_binary_string(binary_string):
    # Converte la stringa binaria in una rappresentazione binaria dei byte
    binary_data = bytes(int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8))
    # Deserializza i dati binari in un oggetto
    output = pickle.loads(binary_data)
    print(output)
	


In [14]:
#SEED TREE
class SeedTree:
    def __init__(self, depth, salt):
        if(depth>0):
            self.seed_m = None
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, seed_m):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            self.seed_m = seed_m
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree(self.seed_m, 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def get_seed_path(self):
        if self.full and self.seed_path is not None:
            return self.seed_path

    def compute_seed_path(self, list_nodes):
        if self.full and len(list_nodes)>0:
            self.seed_path={}
            self.__compute_seed_path(self.depth-1, list_nodes)
            return self.seed_path
        else:
            raise Exception('Albero non creato o foglia non valida')
        
    def get_leaves_from_seed_path(self, seed_path, salt, leaves_indexes):
        return

    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth<=self.depth):
            list_children = self.__create_new_level(list_nodes, depth)
            self.tree.append(list_children)
            self.__make_tree(list_children, depth+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents, '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    #provvisorio
    def __compute_seed_path(self, depth, list_index):
        if depth>0:
            list_parent = sorted( list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))  #prendo gli indici dei nodi padri eliminando i duplicati
            list_sub_root = list(filter(lambda x: ((x % 2 == 0 and x + 1 not in list_parent) or (x % 2 != 0 and x - 1 not in list_parent)), list_parent))   #lista delle 'radici' del livello corrente: per ogni nodo nella lista padri prendo i nodi che non hanno fratelli
            list_new_children = list(filter(lambda x:  x not in list_sub_root, list_parent))     #passo quelli con nodi fratelli come figli per trovare le radici del livello successivo
            if len(list_new_children)>0:
                self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_new_children))
                self.__compute_seed_path(depth-1, list_new_children)
    
    
    def __recompute_leaves_from_partial_root(self, seed):
        return 0
        
            
        

#DA FARE
    def __get_leaves_from_seed_path(self, seed_path, salt):
        return
 

In [15]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2)
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full):
            self.tree = []
            self.tree.append(leaves)
            self.__make_tree(leaves, self.depth-1)
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_merkle_proof(self):
        if self.full and self.merkle_proof is not None:
            return self.merkle_proof

    def compute_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            list_leaves = list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves))
            list_indexes = self.__get_indexes(self.depth, list_leaves, [])
            self.__compute_merkle_proof(list_indexes)
            return self.merkle_proof
        else:
            raise Exception('Albero non creato o foglia non valida')

    def recompute_root(self, merkle_proof, leaves):
        if len(leaves)<self.n_leaves:
            return self.__recompute_root(merkle_proof, leaves)
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth>=0):
            list_parent = self.__create_new_level(list_nodes)
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1)

    def __create_new_level(self, child_nodes):
        parents = []
        for i in range(0, len(child_nodes), 2):
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1])
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __get_indexes(self, depth, list_index, list_proof):
        if(depth > 0):
            list_proof.insert(0, list_index)
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))
            return self.__get_indexes(depth-1, list_parent, list_proof)
        else:
            return list_proof


    def __compute_merkle_proof(self, list_merkle_proofs):
        self.merkle_proof = {}    
        for i in range(len(list_merkle_proofs)): 
            current_proof = list_merkle_proofs[i] #controllo livello i esimo
            if i < len(list_merkle_proofs) - 1:
                next_proof = list_merkle_proofs[i+1]
                current_proof = list(filter(lambda x: x*2 not in next_proof and (x*2)+1 not in next_proof, current_proof))
            if not len(current_proof)==0:
                self.merkle_proof[i+1] = dict(map(lambda x: (x, self.tree[i+1][x]), current_proof))

    def __recompute_root(self, merkle_proof, nodes):
        if len(merkle_proof)==0:
            return nodes
        else:
            index = max(merkle_proof.keys())
            current_level = merkle_proof[index]
            merged_nodes = {**current_level, **nodes}
            merged_nodes = dict(sorted(merged_nodes.items()))
            new_nodes = {}
            for key in merged_nodes:
                new_key = (key + (key % 2)) / 2
                if new_key in parent_dict:
                    new_nodes[new_key] = self.__compute_father_value(new_nodes[new_key],merged_nodes[key])
                else:
                    new_nodes[new_key] = merged_nodes[key]
            del merkle_proof[index]
            return self._recompute_root(self, merkle_proof, new_nodes)

In [16]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_random_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed=seed_e) #e=g^η
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [17]:
#SIGNER
class Signer:
    def __init__(self, seed_sk):
        seed_e , seed_pk = expand_seed(seed_sk) 
        self.V = create_random_matrix(Fp, k, r, seed_pk)
        self.eta = create_random_vector(Fz, n, seed_e)

    def sign_msg(self, msg):
        self.__compute_commitment()
        self.__generate_first_challenge(msg)
        self.__generate_first_response()
        self.__generate_second_challenge()
        self.__generate_second_response()
        return self.__sign()

    def __compute_commitment(self):
        seed_m = generate_random_seed() 
        seed_salt = generate_random_seed()
        self.salt = hash_and_get_seed(seed_salt)
        self.seed_tree = SeedTree(lev_leaf, self.salt)
        self.seed_tree.make_tree(seed_m)
        self.seeds = self.seed_tree.get_leaves()
        list_cmt0 = [] 
        list_cmt1 = ""
        g = get_generator(p, z, Fp)
        for i in range(0, t):
            seed_ui, seed_ei = expand_seed(self.seeds[i])
            eta_i = create_random_vector(Fz, n, seed_ei)
            sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
            v = create_vEn(g, n, Fz, Fp, eta=sigma_i)
            ui = create_random_vector(Fp, n, seed_ui)
            u = component_wise_multiply(v, ui)
            st = u[0:r] + u[r:n]*self.V
            cmt0_i = hexhash(str(st) + str(sigma_i) + str(self.salt) + str(i))
            cmt1_i = hexhash(str(self.seeds[i]) + str(self.salt) + str(i))
            list_cmt0.append(cmt0_i) 
            list_cmt1=list_cmt1+cmt1_i
        #commitment 0
        self.merkle_tree = MerkleTree(t)
        self.merkle_tree.make_tree(list_cmt0)
        c0 = self.merkle_tree.get_root()
        #commitment 1
        #input_c1 = "".join(map(str, list_cmt1))
        c1 = hexhash(list_cmt1) 
        #hash dei due commitment
        self.c01 = hexhash(c0+c1)

    def __generate_first_challenge(self, msg):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch1 = hexhash(msg)+self.c01+hash_salt
        self.seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, self.seed_ch1) 

    def __generate_first_response(self):
        #list_hash_yi=[]
        list_yi=""
        g = get_generator(p, z, Fp)
        for seed_i, ch_i_1 in zip(self.seeds, self.ch1): #t round: rispota yi dipende da seed_i e ch_i_1
            seed_ui, seed_ei = expand_seed(seed_i) 
            ei = create_vEn(g, n, Fz, Fp, seed=seed_ei)
            ui = create_random_vector(Fp, n, seed_ui)
            yi = ui+(ch_i_1)*ei #calcolo risposta i-esima corrispondente all'elemento i-esmino del vettore ch1 
            list_yi+str(yi)
            #hi = hexhash(str(yi))
            #list_hash_yi.append(hi)
        self.h = hexhash(list_yi)

    def __generate_second_challenge(self):#, seed_tree):
        input_ch2 = "".join(self.h)+str(self.seed_ch1)
        self.seed_ch2 = hash_and_get_seed(input_ch2)
        self.ch2 = generate_vector_fixed_hamming_w(w, t, self.seed_ch2)

    def __generate_second_response(self):#CONTROLLARE ORDINE
        index_1, index_0 = [], []
        #questi due vettori sono di lunghezza t-w ovvero i round corrisponenti agli indici in cui il vettore ch2=0
        self.rsp_ch0 = [] # Fp^n x Fz^m
        self.rsp_ch1 = [] # {0,1}^lambda
        g = get_generator(p, z, Fp)
        #
        i = 0
        for seed_i, ch_i_2 in zip(self.seeds, self.ch2):
            if ch_i_2 == 1:#Challenge2 = 1
                index_1.append(i)  
                
            else: #Creo la risposta nel caso challenge2 = 0
                index_0.append(i)

                seed_ui, seed_ei = expand_seed(seed_i)
                #ricalcolo sigma i
                eta_i = create_random_vector(Fz, n, seed_ei)
                sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
                #ricalcolo yi
                ei = create_vEn(g, n, Fz, Fp, seed_ei)
                ui = create_random_vector(Fp, n, seed_ui) 
                yi = ui+(self.ch1[i])*ei
                
                #risposta challenge 0
                self.rsp_ch0.append([yi, sigma_i]) 

                #risposta challenge 1
                cmt1_i = hexhash( str(seed_i) + str(self.salt) + str(i) )
                
                self.rsp_ch1.append(cmt1_i)

            i = i+1
            
        self.seed_path = self.seed_tree.compute_seed_path(index_1)
        self.merkle_proof = self.merkle_tree.compute_proof(index_0)

    def __sign(self):
        signature = {}
        signature["salt"] = self.salt
        signature["c01"] = self.c01
        signature["ch2"] = self.seed_ch2
        signature["seed_path"] = self.seed_path
        signature["merkle_proof"] = self.merkle_proof
        signature["rsp_ch0"] = self.rsp_ch0
        signature["rsp_ch1"] = self.rsp_ch1
        binary_signature = convert_object_to_binary_string(signature)
        return binary_signature

In [18]:
#VERIFIER
class Verifier:

    def check_sign(self, sign):
        self.__retrieve_sign(sign)
        #self.msg = msg
        
    def retrive_challenges(self):
        hash_salt = convert_int_to_hash_string(self.salt)
        hash_msg = hexhash(self.msg)
        #challenge1
        input_ch1 = hash_msg+self.c01+hash_salt
        seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, seed_ch1)
        #challenge2
        self.ch2 = generate_vector_fixed_hamming_w(w, t, self.seed_ch2)
        #recupero indici
        #self.index_0 = []
        #self.index_1 = []
        #[self.index_1.append(i) if x == 1 else self.index_0.append(i) for i, x in enumerate(self.ch2)]
        index_1 = []
        for i, x in enumerate(self.ch2):
            if x == 1: index_1.append(i)
            
        merkle_tree = MerkleTree(t)
        #i seeds servono solo quando ch_2 = 1 
        self.seeds = merkle_tree.get_leaves_from_seed_path(self.seed_path, self.salt, index_1)
    
    def recompute_responses(self):
        g = get_generator(p, z, Fp)
        c0={}
        list_cmt1=""
        list_y=""
        j=0
        for i, x in enumerate(self.ch2):
            if x ==1:
                seed_ui, seed_ei = expand_seed(self.seeds[i]) 
                #RICALCOLO RISPOSTA CHALLENGE 1
                ei = create_vEn(g, n, Fz, Fp, seed=seed_ei)
                ui = create_random_vector(Fp, n, seed_ui)   
                yi = ui+(self.ch1[i])*ei
                list_y=list_y+yi
                #RICALCOLO COMMITMENT 1
                #cmt1_i = hexhash(ui, ei, salt, i)
                cmt1_i = hexhash(str(self.seeds[i]) + str(self.salt) + str(i))
                list_cmt1=list_cmt1+cmt1_i
                #hi = hexhash(str(yi))
            else:
                [yi, sigma_i]=self.rsp_ch0[j]
                #verificare sigma_i appartenga a G

                #RICALCOLO SIGMA TILDE 
                v = create_vEn(g, n, Fz, Fp, eta=sigma_i)
                yi_ = component_wise_multiply(v, yi)
                st = yi_[0:r] + yi_[r:n]*self.V - self.ch1[i]*self.s

                #RICALCOLO COMMITMENT 0 
                cmt0_i = hexhash(str(st)+str(sigma_i)+str(self.salt)+str(i))
                c0[i]=cmt0_i

                #ESTRAGGO COMMITMENT 1
                cmt1_i = self.rsp_ch1[j]
                list_cmt1=list_cmt1+cmt1_i
                j = j+1

        merkle_tree = MerkleTree(t)
        recomputed_root = merkle_tree.recompute_root(self.merkle_proof, c0)
        recomputed_c1 = hexhash(list_cmt1) 
        recomputed_c01 = hexhash(recomputed_root+recomputed_c1)
        recomputed_h = hexhash(list_y)
        #h = hexhash(h1,...,hn)
        #c0 = hexhash(c01,...,c0t)
        #VerifyMerkleProof   

    def __retrieve_sign(self, sign):
        signature = get_object_from_binary_string(sign)
        self.salt = signature["salt"]
        self.c01 = signature["c01"]
        self.seed_ch2 = signature["ch2"] 
        self.seed_path = signature["seed_path"]
        self.merkle_proof = signature["merkle_proof"]
        self.rsp_ch0 = signature["rsp_ch0"]
        self.rsp_ch1 = signature["rsp_ch1"]


In [27]:
seed_sk, seed_pk, s = key_gen()
signer = Signer(seed_sk)
signature = signer.sign_msg("buonasera")

In [29]:
for x in signer.seed_path.keys():
    print(x)
    print(signer.seed_path[x].keys())


6
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 56, 57, 58, 59, 60, 61])
5
dict_keys([0, 1, 2, 3, 6, 7, 8, 9, 12, 13, 16, 17, 20, 21, 22, 23, 24, 25, 28, 29])
4
dict_keys([0, 1, 10, 11])


In [ ]:

#verifier = Verifier()
#verifier.check_sign(signature)
#Salt || d01 || db || MerkleProofs || SeedPath || rsp_ch0 || rsp_ch1